 currently this document is executed and use to create a report.
 It should continue to be a report and code at the same time.
 The notebook is the reporting and coding interface.

 The best case scenario is that this document gets copied and executed somewhere returned with a new file name because the execution is updated.

 Effectively this will have Covid test and documentation all in the same place

# gathering thingiverse things

an effective scraping and caching strategy for gathering information on thingiverse.

each activity requires:
1. making requests
2. tidying requests
3. creating tasks to save salient data, create a file witht he missing columsn


1. gather the index of things from search
2. save the index to disk as parquet
3. read the index of things from disks
4. save the things to disc

the second request is solely to get the description information about the thing cause that is not included in the search response
despite the openapi schema saying that it should be


In [12]:
with __import__("importnb").Notebook():
    from app import App as App
import os, time, random, operator
from pandas import Series, DataFrame

In [13]:
@App.impl
def set_params(params, app):
    params |= dict(access_token=os.environ["THINGIVERSE_ACCESS_TOKEN"])

In [14]:
def search_client(app: App=None, **kwargs):
    return (app.cache / "thingiverse" / "search_responses").client(**kwargs)

def thing_client(app: App=None, **kwargs):
    return (app.cache / "thingiverse" / "thing_responses").client(**kwargs)

`search_tags` 

term
: the escaped search term

pages
: the maximum number of pages to visit

per_page
: the maximum number of results returned by api

In [15]:
    @App.impl(specname="get_responses", tryfirst=True)
    def search_tags(app: App, term="assistive+technology", pages=None, per_page=30) -> None:
        page = 1
        requests = []
        while not pages or page <= pages:
            response = search_client(app).get(
                f"https://api.thingiverse.com/search/{term}", 
                dict(per_page=per_page, page=page, type="thing") | app.params
            )
            
            requests.append(response.url)
            if app.verbosity:
                print(requests[-1])
            if pages is None:
                pages = (response.json()["total"] // per_page) + 1
            page += 1
        return requests

In [16]:
    @App.impl(specname="get_frames")
    def get_search_results(app: App) -> DataFrame:
        return (
            Series((client := search_client(app)).cache.urls())
            .apply(client.get)
            .apply(operator.methodcaller("json"))
            .apply(lambda x: x.get("hits", []))
            .explode()
            .apply(Series)
            .set_index("public_url")
        )

In [17]:
    @App.impl(specname="get_responses", trylast=True)
    def get_things(app: App):
        results = []
        client = thing_client(app)
        for j, (i, row) in enumerate(
            get_search_results(app).dropna(subset="id").iterrows()
        ):
            if (row.url + "?access_token=REDACTED") not in client.cache.urls():
                time.sleep(random.random()*2)
            results.append(thing_client(app).get(row.url, params=app.params).url)
            if app.verbosity:
                print(results[-1].partition("?")[0])
        return results

In [18]:
    def get_thing_details_tidy(df, index):
        df = df[df.columns.difference(index.columns)]
        df = df.join(
            df.pop("zip_data").dropna().apply(Series)
            .stack().explode().dropna().apply(operator.itemgetter("url"))
            .groupby(level=0).agg(list).rename("files")
        )
        # the default image is in the files
        df.pop("default_image");
        df.pop("edu_details_parts");
        df.pop("education");
        df.pop("description_html");
        # TODO there is information in details_parts than can be extracted for search.
        details_parts = df.pop("details_parts").explode().apply(Series).set_index("type", append=True).fillna("")
        # detail_parts is segmented string info we cram into  a single block
        details_parts = details_parts.join(
            details_parts.apply("\n".join, axis=1).rename("parts")
        )
        details_parts = details_parts.reset_index(1, drop=True)
        df = df.join(
            (("<h2>" + details_parts["name"] + "</h2>") + "\n" + details_parts["parts"]).dropna().groupby(level=0).agg("\n".join).rename("details_parts")
        )
        df["ancestors"] = df.ancestors.explode().dropna().apply(Series).public_url.groupby(level=0).agg(list)
        return df

In [19]:
@App.impl
def get_frames(app):
    things = (
        Series(thing_client(app).cache.responses)
        .apply(operator.methodcaller("json"))
        .apply(Series)
    ).set_index("url")

    index = (
        Series(search_client(app).cache.responses)
        .apply(operator.methodcaller("json"))
        .apply(Series)
        .explode("hits")["hits"]
        .apply(Series)
    ).set_index("url")
    return things.join(index[index.columns.difference(things.columns)])

In [20]:
if __name__ == "__main__":
    app = App()
    app.main(__import__(__name__), run=0 or locals().get("__file__") is not None)
    df = app.compact()
    display(df.T.fillna(""))
        

,https://www.thingiverse.com/thing:1852950,https://www.thingiverse.com/thing:1103082,https://www.thingiverse.com/thing:1871831,https://www.thingiverse.com/thing:2745606,https://www.thingiverse.com/thing:3057298,https://www.thingiverse.com/thing:2785602,https://www.thingiverse.com/thing:5936465,https://www.thingiverse.com/thing:242639,https://www.thingiverse.com/thing:2801872,https://www.thingiverse.com/thing:1104871,...,https://api.thingiverse.com/things/3699364,https://api.thingiverse.com/things/3699364,https://api.thingiverse.com/things/2969609,https://api.thingiverse.com/things/2863119,https://api.thingiverse.com/things/7092470,https://api.thingiverse.com/things/6652475,https://api.thingiverse.com/things/2870410,https://api.thingiverse.com/things/3086651,https://api.thingiverse.com/things/4814288,https://api.thingiverse.com/things/2246592
id,1852950,1103082,1871831,2745606,3057298,2785602,5936465,242639,2801872,1104871,...,3699364,3699364,2969609,2863119,7092470,6652475,2870410,3086651,4814288,2246592
name,Open Assistive Technology - Key Turner,Easy Zipper for Assistive Technology Challenge,Open Assistive Technology - Bag Holder,Assistive Technology Mouse (ATMouse),Assistive Technology - Bag Holder,Assistive Technology: Parametric Ring Pen Holder,Plug Puller - Assistive Technology Solution fr...,Flexy-Hand,Assistive Technology Adapted Dice Roller,Mouth Cable Bobbin for Assistive Technology Ch...,...,Smartphone holder for magnifying,Smartphone holder for magnifying,OT student project : hand guide for lock / gui...,Gyrospoon v0.6.1,Handcycle Grips Toolkit for Quadriplegic or Pa...,Plug Puller - Velcro Strap Version [Use With C...,Volksswitch - the People's AT Switch (Version 1),LAMP 12x7-Hole KeyGuard for iPad model A1822,Maker Vikings Assistive Tech (Make:able Challe...,Thumb Prosthesis
url,https://api.thingiverse.com/things/1852950,https://api.thingiverse.com/things/1103082,https://api.thingiverse.com/things/1871831,https://api.thingiverse.com/things/2745606,https://api.thingiverse.com/things/3057298,https://api.thingiverse.com/things/2785602,https://api.thingiverse.com/things/5936465,https://api.thingiverse.com/things/242639,https://api.thingiverse.com/things/2801872,https://api.thingiverse.com/things/1104871,...,,,,,,,,,,
created_at,2016-10-27T01:46:07+00:00,2015-10-31T12:20:01+00:00,2016-11-05T01:50:01+00:00,2018-01-04T07:53:20+00:00,2018-08-20T13:49:03+00:00,2018-02-07T18:50:42+00:00,2023-03-29T19:04:53+00:00,2014-03-04T17:12:57+00:00,2018-02-21T16:34:13+00:00,2015-11-01T20:29:13+00:00,...,2019-06-18T18:06:27+00:00,2019-06-18T18:06:27+00:00,2018-06-20T08:09:31+00:00,2018-04-15T15:25:00+00:00,2025-07-15T17:18:50+00:00,2024-06-07T06:24:07+00:00,2018-04-20T02:55:01+00:00,2019-08-13T14:23:51+00:00,2021-04-02T04:02:57+00:00,2017-04-13T14:25:48+00:00
thumbnail,https://cdn.thingiverse.com/renders/11/e3/d6/0...,https://cdn.thingiverse.com/renders/ef/fa/1a/f...,https://cdn.thingiverse.com/renders/2a/55/85/8...,https://cdn.thingiverse.com/renders/1c/31/d9/b...,https://cdn.thingiverse.com/renders/21/c1/62/1...,https://cdn.thingiverse.com/renders/d1/ff/89/a...,https://cdn.thingiverse.com/assets/66/86/13/ac...,https://cdn.thingiverse.com/renders/2f/34/ae/1...,https://cdn.thingiverse.com/renders/af/b2/9d/a...,https://cdn.thingiverse.com/renders/20/56/6f/5...,...,https://cdn.thingiverse.com/renders/ed/26/d5/3...,https://cdn.thingiverse.com/renders/ed/26/d5/3...,https://cdn.thingiverse.com/renders/bb/04/02/d...,https://cdn.thingiverse.com/renders/f7/fe/2a/5...,https://cdn.thingiverse.com/assets/38/ba/14/ea...,https://cdn.thingiverse.com/assets/10/65/ff/63...,https://cdn.thingiverse.com/renders/6b/26/13/0...,https://cdn.thingiverse.com/renders/d6/09/99/2...,https://cdn.thingiverse.com/assets/83/17/6e/b2...,https://cdn.thingiverse.com/renders/14/7d/29/c...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
root_comment_count,,,,,,,,,,,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,3.0
is_derivative,,,,,,,,,,